In [1]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

In [ ]:
class MLP_500_500_500_classifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Linear(22, 500)  
        self.relu = nn.ReLU()                          
        self.hidden_layer_relu = nn.Sequential(
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU()
        )
        self.layer2 = nn.Linear(500, 30)

    def forward(self, x):

        x = self.relu(self.layer1(x))
        x = self.hidden_layer_relu(x)
        logits = self.layer2(x)
        
        return logits

In [3]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

y_best_ansatz = np.argmax(y, axis=1)  

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [4]:
print(y_best_ansatz)

[ 6  0  6  0 11 14  0 14 14  0  0  2  0  6  6 14  2  6  0 11  2 14 14 16
  2  6  6  2 11  6 14  6 16 14 14 14 14  8 14  6  6 14  6 14  6  8 14  6
  6  6 14  6  6  6  6  6  8  6 14  0  6  6  6  6  6  1  6  6  6  2  6  6
  6  6  6  6  6  2  2  6  6  6  6  6  6  6  2  6  2  6 13  0 12  0  7  7
  7  8  0  7  7 12  7  8 21 12 13  7  7  7 12 17  7 13 21  7  7  7  6  7
  6 14  6 25  6  6  1 14  1  6  6  6  6 14 16  6  6  6  6  6  6  6 14  1
  6  6  6 14  6 14  6  6 14 14 14  6 14 14  6  6  6  6 14  6  6  6  6  6
 14  6  6  6  6 14  6  6  6  6  6  6 19 20  3 22 11 19  1  6 20 29 20  0
  0 19 28 29  6 11  6 27  7 17 20 26  0 22 17  3  0 29 13  2 27  0  7 10
 24 24 20  6 10 22 24 22 10 10 14  8 15 17 10 22  6  0 11  2 12 22 27 20
  2  0 15  0 29 10 12  2 17  6  3 29 16  0 11  0  0  7  7  0  6 10 14  0
  6  6  0  3 10  0  2 11 16 18  2 18 29  6 13 16 16 16 22  0 16 23  6 16
 20 18 18 16 16  0 11 16 16 17 16 20  6  8  3 11 11  6 22  3 22  0 29 11
  0 20  2  6 22 28 29  0 12 27 11 26 29 27 28  0 12

In [5]:
print(y_best_ansatz.shape)  # Deve ser (N,), não (N, 1)

(360,)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [7]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [8]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000):
    model.to(device)
    early_stopping = EarlyStopping(patience=10, min_delta=0.0)
    best_model = None
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_val = y_val.squeeze()  # Garante que y_val é 1D
                
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()  
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)
        
        
        val_acc = correct / total
        val_loss /= len(val_loader)
        train_loss /= len(train_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_acc

In [9]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_best_ansatz[train_idx], y_best_ansatz[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_500_500_500_classifier()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization
    
    # Treinar fold
    best_model, best_acc = train_fold(model, train_loader, val_loader, criterion, optimizer)
    results.append(best_acc)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-500-500-500-classifier_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f"Acurácia média: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"Acurácias por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 3.1831 - Val Loss: 3.1011 - Val Acc: 0.3000
Epoch 2/1000000 - Train Loss: 2.6824 - Val Loss: 2.8337 - Val Acc: 0.3000
Epoch 3/1000000 - Train Loss: 2.4644 - Val Loss: 2.7462 - Val Acc: 0.2583
Epoch 4/1000000 - Train Loss: 2.3961 - Val Loss: 2.8082 - Val Acc: 0.2583
Epoch 5/1000000 - Train Loss: 2.2737 - Val Loss: 2.6794 - Val Acc: 0.2583
Epoch 6/1000000 - Train Loss: 2.2588 - Val Loss: 2.7278 - Val Acc: 0.2583
Epoch 7/1000000 - Train Loss: 2.2696 - Val Loss: 2.6969 - Val Acc: 0.2583
Epoch 8/1000000 - Train Loss: 2.2331 - Val Loss: 2.7374 - Val Acc: 0.2750
Epoch 9/1000000 - Train Loss: 2.2311 - Val Loss: 2.7614 - Val Acc: 0.2583
Epoch 10/1000000 - Train Loss: 2.2494 - Val Loss: 2.7618 - Val Acc: 0.2750
Epoch 11/1000000 - Train Loss: 2.2450 - Val Loss: 2.6781 - Val Acc: 0.2583
Epoch 12/1000000 - Train Loss: 2.2291 - Val Loss: 2.6742 - Val Acc: 0.3000
Epoch 13/1000000 - Train Loss: 2.1804 - Val Loss: 2.7693 - Val Acc: 0.2750
Epoch 14/1000000

In [10]:
model = MLP_500_500_500_classifier().to(device)
#carregando o melhor fold treinado
model.load_state_dict(torch.load('.//models_salvos/best_model_MLP-500-500-500-classifier_fold2.pt'))  # Substitua pelo caminho correto
model.eval()  # Modo de avaliação

MLP_500_500_500_classifier(
  (layer1): Linear(in_features=22, out_features=500, bias=True)
  (relu): ReLU()
  (hidden_layer_relu): Sequential(
    (0): Linear(in_features=500, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): ReLU()
  )
  (layer2): Linear(in_features=500, out_features=30, bias=True)
)

In [11]:
input = X[6]
input_tensor = torch.FloatTensor(input).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    print(logits)
    print(logits.shape)
    probabilities = torch.softmax(logits, dim=0)  # Shape (1, 30)

tensor([ 1.7966,  0.3270,  1.7380, -2.6257, -5.5529, -5.4489,  2.3973,  1.6526,
         0.5597, -3.6324, -2.3316,  0.8602,  0.8030,  0.8055,  2.1152, -5.2260,
         0.7682, -2.6002, -3.1180, -2.7146, -2.3959,  0.2236, -2.4988, -3.0993,
        -2.4608, -0.1707, -2.9682, -2.6273, -3.8079, -2.6318])
torch.Size([30])


In [12]:
print(probabilities)

tensor([1.1740e-01, 2.7003e-02, 1.1071e-01, 1.4095e-03, 7.5477e-05, 8.3748e-05,
        2.1407e-01, 1.0165e-01, 3.4078e-02, 5.1506e-04, 1.8915e-03, 4.6023e-02,
        4.3467e-02, 4.3576e-02, 1.6144e-01, 1.0466e-04, 4.1979e-02, 1.4459e-03,
        8.6157e-04, 1.2896e-03, 1.7738e-03, 2.4351e-02, 1.6002e-03, 8.7776e-04,
        1.6623e-03, 1.6416e-02, 1.0007e-03, 1.4073e-03, 4.3217e-04, 1.4009e-03])


In [13]:
# Obter índices e valores das top-k probabilidades (ex.: top-5)
k = 3
top_k_probs, top_k_indices = torch.topk(probabilities, k=k, dim=0)

# Converter para numpy e exibir
top_k_probs = top_k_probs.cpu().numpy().flatten()
top_k_indices = top_k_indices.cpu().numpy().flatten()

print("Melhores ansatzes e suas probabilidades:")
for i, (idx, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    print(f"{i+1}º: Ansatz {idx} - Probabilidade: {prob:.4f}")

Melhores ansatzes e suas probabilidades:
1º: Ansatz 6 - Probabilidade: 0.2141
2º: Ansatz 14 - Probabilidade: 0.1614
3º: Ansatz 0 - Probabilidade: 0.1174
